In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from dataclasses import dataclass

In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

Mounted at /content/gdrive


In [ ]:
import cv2
data_dir = '/content/gdrive/MyDrive/MODEL/data_new_modified/Train'
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2690 images belonging to 31 classes.
Found 657 images belonging to 31 classes.


In [4]:
def cnn_model(input_shape=(32, 32, 3)):

    model = Sequential()

    #------------------------------------
    # Conv Block 1: 32 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #------------------------------------
    # Conv Block 2: 64 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #------------------------------------
    # Conv Block 3: 64 Filters, MaxPool.
    #------------------------------------
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #------------------------------------
    # Flatten the convolutional features.
    #------------------------------------
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(31, activation='softmax'))

    return model

In [5]:
model = cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                   

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=1e-4,
                               patience=10,
                               verbose=2,
                               restore_best_weights=True)

save_weights = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join('/content/gdrive/MyDrive/CS231', 'CNN.h5'),
        save_best_only=True,
        save_weights_only=True,
        monitor='val_loss',
        mode='min',
        verbose=2
    )

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
             )

In [ ]:
history = model.fit(train_data,
                    batch_size=32,
                    epochs=100,
                    verbose=1,
                    validation_data=val_data,callbacks=[early_stopping,save_weights]
                   )

Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 1.0565 - accuracy: 0.7186
Epoch 1: val_loss improved from inf to 0.14885, saving model to /content/gdrive/MyDrive/CS231/CNN.h5
85/85 [==============================] - 759s 9s/step - loss: 1.0565 - accuracy: 0.7186 - val_loss: 0.1489 - val_accuracy: 0.9513
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.1629 - accuracy: 0.9513
Epoch 2: val_loss improved from 0.14885 to 0.14493, saving model to /content/gdrive/MyDrive/CS231/CNN.h5
85/85 [==============================] - 5s 53ms/step - loss: 0.1629 - accuracy: 0.9513 - val_loss: 0.1449 - val_accuracy: 0.9574
Epoch 3/100
85/85 [==============================] - ETA: 0s - loss: 0.0989 - accuracy: 0.9703
Epoch 3: val_loss improved from 0.14493 to 0.09870, saving model to /content/gdrive/MyDrive/CS231/CNN.h5
85/85 [==============================] - 5s 53ms/step - loss: 0.0989 - accuracy: 0.9703 - val_loss: 0.0987 - val_accuracy: 0.9817
Epoch 4/100
85

In [6]:
test_path = '/content/gdrive/MyDrive/MODEL/data_new_modified/Test'

In [7]:
index_to_class = {0: '0',1: '1',2: '2',3: '3',4: '4',5: '5',6: '6',7: '7',8: '8',9: '9',10: 'A',11: 'B',12: 'C',13: 'D',14: 'E',15: 'F',16: 'G',17: 'H',18: 'K',
 19: 'L',
 20: 'M',
 21: 'N',
 22: 'P',
 23: 'R',
 24: 'S',
 25: 'T',
 26: 'U',
 27: 'V',
 28: 'X',
 29: 'Y',
 30: 'Z'}

In [8]:
model.load_weights('/content/gdrive/MyDrive/CS231/CNN.h5')

In [9]:
import cv2
total = []
right =[]
for class_name in os.listdir(test_path):
  class_path = os.path.join(test_path,class_name)
  for img_name in os.listdir(class_path):
    total.append(class_name)
    img_path = os.path.join(class_path,img_name)
    img = cv2.imread(img_path,1)
    img = cv2.resize(img,(32,32))
    img = np.array(img)
    img = np.expand_dims(img,axis=0)
    img = img / 255.0
    pred = model.predict(img)
    pred = (np.argmax(pred,axis=1))
    pred_2 = index_to_class[pred[0]]
    right.append(pred_2)

1/1 [==============================] - 0s 22ms/step


In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(total,right)

0.9173333333333333

In [11]:
from sklearn.metrics import precision_score
precision_score(total,right,average='macro')

0.930894161183535

In [12]:
from sklearn.metrics import recall_score
recall_score(total,right,average='macro')

0.9240591397849462

In [13]:
from sklearn.metrics import f1_score
f1_score(total,right,average='macro')

0.9174820333805491